In [1]:
#importing neccesary libraries to train our sentiment analysis model.
import numpy as np # linear algebra
import pandas as pd # data processing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import torch
import transformers

from datasets import Dataset,DatasetDict

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import TrainingArguments,Trainer

from scipy.special import softmax

/kaggle/input/airlinereviews-twitter/train.csv
/kaggle/input/airlinereviews-twitter/test.csv


2024-04-07 04:20:46.800554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 04:20:46.800671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 04:20:46.920164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#Reading the required files for training and testing data.
train_df = pd.read_csv("/kaggle/input/airlinereviews-twitter/train.csv")
test_df = pd.read_csv("/kaggle/input/airlinereviews-twitter/test.csv")

# Prepare the Dataset

We can see the below dataset already available to us. The following columns are irrelevant for our usecase 'airline_sentiment_gold', 'name', 'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'



In [3]:
train_df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [4]:
train_df.columns, test_df.columns

(Index(['tweet_id', 'airline_sentiment', 'airline', 'airline_sentiment_gold',
        'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
        'tweet_created', 'tweet_location', 'user_timezone'],
       dtype='object'),
 Index(['tweet_id', 'airline', 'text', 'sentiment'], dtype='object'))

In [5]:
for i in test_df.index: #changing sentiment labels to maintain uniformity
    test_df.loc[i, 'sentiment'] = test_df.loc[i, 'sentiment']-1
    
test_df.rename(columns = {'sentiment':'labels'}, inplace = True)

Dropping the irrelevant columns to make our analysis hassle-free :

In [6]:
train_df.drop(columns=['airline_sentiment_gold', 'name', 'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'], inplace = True)

In [7]:
train_df.columns, test_df.columns

(Index(['tweet_id', 'airline_sentiment', 'airline', 'text'], dtype='object'),
 Index(['tweet_id', 'airline', 'text', 'labels'], dtype='object'))

Renaming columns for ease of use:

In [8]:
train_df.rename(columns = {'airline_sentiment':'sentiment'}, inplace = True) 

Our datasets look like:

In [9]:
train_df.head()

,tweet_id,sentiment,airline,text
0,567900433542488064,negative,Southwest,"@SouthwestAir I am scheduled for the morning, ..."
1,569989168903819264,positive,Southwest,@SouthwestAir seeing your workers time in and ...
2,568089179520954368,positive,United,@united Flew ORD to Miami and back and had gr...
3,568928195581513728,negative,Southwest,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,568594180014014464,negative,United,@united so our flight into ORD was delayed bec...


In [10]:
test_df = test_df.astype({'labels':int})

# Analyze the Datasets - Essential to make good Training set, Validation set, and Testing set.
What are the elements in the datasets? How are they distributed?

In [11]:
print(train_df.airline.unique(), train_df.sentiment.unique())

['Southwest' 'United' 'Delta' 'US Airways' 'American' 'Virgin America'] ['negative' 'positive' 'neutral']


We have review tweets mentioning 6 unique airlines, and the the reviews are labeled with 3 sentiments ( 'negative', 'positive', 'neutral' )

How are they distributed ?

1. Distribution with respect to sentiments
2. Distribution with respect to Airlines

## Observing train_df

Sentiment distribution

In [12]:
print('Positive -', train_df.loc[train_df['sentiment']=='positive'].shape[0])
print('Neutral -', train_df.loc[train_df['sentiment']=='neutral'].shape[0])
print('Negative -', train_df.loc[train_df['sentiment']=='negative'].shape[0])

Positive - 1802
Neutral - 2327
Negative - 6851


Airline distribution

In [13]:
for airline in train_df.airline.unique():
    print(f'{airline} -', train_df.loc[train_df['airline']==airline].shape[0])

Southwest - 1817
United - 2928
Delta - 1639
US Airways - 2152
American - 2078
Virgin America - 366


Observations - 
- Number of negative tweets are a lot more than positive and neutral tweets.
- Number of tweets mentioning Virgin America are very less compared to other airlines.

## Observing test_df

Airline distribution ( Sentiments aren't available as this is a test set )

In [14]:
for airline in test_df.airline.unique():
    print(f'{airline} -', test_df.loc[test_df['airline']==airline].shape[0])

American - 167
Southwest - 165
US Airways - 175
United - 219
Delta - 131
Virgin America - 42


Observations -
- Test dataset is a scaled down version of Training dataset

# Making Validation Set

What is a validation set? How will we make our validation dataset? 
The validation set, sometimes called the development set, is an intermediary between the training and test sets. Its primary purpose is to fine-tune the model's hyperparameters and assess its performance during training. By evaluating the model's performance on the validation set, we can make informed decisions about adjusting hyperparameters, selecting the best-performing model, or identifying potential overfitting or underfitting issues. We want the Validation set to reflect the data we will be getting in real world. We will be making Validation dataset using tweets of one specific airline. This should give a good view of future tweets we have to perform on. We avoid overfitting airline-specific tweet trends.

Usually the training and validation split is 80-20. Lets choose our validation set.

In [15]:
sum_temp=0
for airline in train_df.airline.unique():
    print(f'{airline} -', train_df.loc[train_df['airline']==airline].shape[0])
    sum_temp += train_df.loc[train_df['airline']==airline].shape[0]
    
print(f'Total tweets -', sum_temp)
print(f'20% percent of Total', 0.2*sum_temp)

Southwest - 1817
United - 2928
Delta - 1639
US Airways - 2152
American - 2078
Virgin America - 366
Total tweets - 10980
20% percent of Total 2196.0


The validation set has to be around 2100-2200 tweets. We will be doing 3-cross validation.
1. Tweets mentioning United are going to be validation set, rest will be used for training.
2. Tweets mentioning US Airways are going to be validation set, rest will be used for training.
3. Tweets mentioning American are going to be validation set, rest will be used for training.

Before splitting the data to different sets for validation and training sets, apply `preprocess(row)` and `labelling(row)` to the dataframe, and also rename the sentiment column to 'labels'.

1. What is `preprocess(row)` ?

Ans - This function takes the text in the passed row and returns text with the following changes-
    1. @AirwayName is masked to @user, to avoid biases related to certain airways. 
    2. masks links into the string 'http' to avoid irrelevant link characters.

2. What is `labelling(row)` ? 

Ans - This function changes the sentiment row of our dataset. It takes in string 'positive', 'neutral', and 'negative' and returns integer 2, 1, and 0 respectively. This is needed for finetuning our model.

In [16]:
def preprocess(row):
    text = row['text']
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def labelling(row):
    sentiment = row['sentiment']
    if(sentiment == 'neutral'): return 1
    elif (sentiment == 'positive'): return 2;
    else: return 0

In [17]:
train_df['text'] = train_df.apply(preprocess, axis=1)
train_df['sentiment'] =  train_df.apply(labelling, axis=1)
train_df.rename(columns = {'sentiment':'labels'}, inplace = True)

In [18]:
train_split_united = train_df.loc[train_df['airline'] != 'United']
test_split_united = train_df.loc[train_df['airline'] == 'United']

In [19]:
train_split_USAirways = train_df.loc[train_df['airline'] != 'US Airways']
test_split_USAirways = train_df.loc[train_df['airline'] == 'US Airways']

In [20]:
train_split_american = train_df.loc[train_df['airline'] != 'American']
test_split_american = train_df.loc[train_df['airline'] == 'American']

To pass this data to Huggingface transformers, these need to be converted to Datasets

In [21]:
ds_united = Dataset.from_pandas(train_split_united)
eval_ds_united = Dataset.from_pandas(test_split_united)

In [22]:
ds_USAirways = Dataset.from_pandas(train_split_USAirways)
eval_ds_USAirways = Dataset.from_pandas(test_split_USAirways)

In [23]:
ds_american = Dataset.from_pandas(train_split_american)
eval_ds_american = Dataset.from_pandas(test_split_american)

We are done with preparing our datasets, now its time to finetune and test.

# Finetuning


In [24]:
model_nm = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokz = AutoTokenizer.from_pretrained(model_nm)

def tokFunc(x): return tokz(x['text'])

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [25]:
tok_ds_united = ds_united.map(tokFunc, batched=True)
tok_eval_ds_united = eval_ds_united.map(tokFunc, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [26]:
tok_ds_USAirways = ds_USAirways.map(tokFunc, batched=True)
tok_eval_ds_USAirways = eval_ds_USAirways.map(tokFunc, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [27]:
tok_ds_american = ds_american.map(tokFunc, batched=True)
tok_eval_ds_american = eval_ds_american.map(tokFunc, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [28]:
bs = 64
epochs = 2

In [29]:
args = TrainingArguments(
    output_dir='./kaggle/working/results_united',                   # output directory
    num_train_epochs=epochs,                  # total number of training epochs
    per_device_train_batch_size=bs,   # batch size per device during training
    per_device_eval_batch_size=bs,    # batch size for evaluation
    warmup_steps=100,                          # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                        # strength of weight decay
    logging_dir='./kaggle/working/logs_united',                     # directory for storing logs
    logging_steps=160,                         # when to print log
    evaluation_strategy='steps',              # evaluate every n number of steps.
    eval_steps=160,                            # how often to evaluate. If not set defaults to number of logging_steps
    load_best_model_at_end=True,              # to load or not the best model at the end
    save_steps=160,                            # create a checkpoint every time we evaluate,
    seed=42,                             # seed for consistent results
)

num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=num_labels)

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly 

In [30]:
!pip install evaluate

import evaluate

metric = evaluate.load("accuracy")

#making a function thats used to calculate the metrics while trainer finetunes the model

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00a 0:00:01


## For United Split

In [31]:
'''trainer = Trainer(model, args, train_dataset=tok_ds_united, eval_dataset=tok_eval_ds_united,
                  tokenizer=tokz, compute_metrics=compute_metrics)'''

'trainer = Trainer(model, args, train_dataset=tok_ds_united, eval_dataset=tok_eval_ds_united,\n                  tokenizer=tokz, compute_metrics=compute_metrics)'

In [32]:
'''import gc

torch.cuda.empty_cache()'''

'import gc\n\ntorch.cuda.empty_cache()'

In [33]:
#trainer.train()

In [34]:
#trainer.save_model("/kaggle/working/finetuned")

In [35]:
#model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/finetuned")

In [36]:
#test_split_united['pred']=-1
#test_split_united

In [37]:
#test_split_united['pred']=-1

In [38]:
'''encoded_input = [tokz(x, return_tensors='pt') for x in test_split_united['text']]
model.to("cpu")
j=0
indices=test_split_united.index
for i in encoded_input:
    output=model(**i)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    test_split_united.loc[indices[j], "pred"] = np.argmax(scores)
    j+=1;
    print('i', end ="")'''

'encoded_input = [tokz(x, return_tensors=\'pt\') for x in test_split_united[\'text\']]\nmodel.to("cpu")\nj=0\nindices=test_split_united.index\nfor i in encoded_input:\n    output=model(**i)\n    scores = output[0][0].detach().numpy()\n    scores = softmax(scores)\n    test_split_united.loc[indices[j], "pred"] = np.argmax(scores)\n    j+=1;\n    print(\'i\', end ="")'

In [39]:
#test_split_united.to_csv("/kaggle/working/test_split_united.csv")

In [40]:
#for i in range(3):
#    print(test_split_united.loc[(test_split_united['pred']==1)].shape[0])

## For USAirways Split

In [41]:
'''trainer = Trainer(model, args, train_dataset=tok_ds_USAirways, eval_dataset=tok_eval_ds_USAirways,
                  tokenizer=tokz, compute_metrics=compute_metrics)
trainer.train()'''

'trainer = Trainer(model, args, train_dataset=tok_ds_USAirways, eval_dataset=tok_eval_ds_USAirways,\n                  tokenizer=tokz, compute_metrics=compute_metrics)\ntrainer.train()'

In [42]:
#trainer.save_model("/kaggle/working/finetuned")

In [43]:
'''model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/finetuned")
test_split_USAirways['pred']=-1
encoded_input = [tokz(x, return_tensors='pt') for x in test_split_USAirways['text']]
model.to("cpu")
j=0
indices=test_split_USAirways.index
for i in encoded_input:
    output=model(**i)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    test_split_USAirways.loc[indices[j], "pred"] = np.argmax(scores)
    j+=1;
    print('i', end ="")'''

'model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/finetuned")\ntest_split_USAirways[\'pred\']=-1\nencoded_input = [tokz(x, return_tensors=\'pt\') for x in test_split_USAirways[\'text\']]\nmodel.to("cpu")\nj=0\nindices=test_split_USAirways.index\nfor i in encoded_input:\n    output=model(**i)\n    scores = output[0][0].detach().numpy()\n    scores = softmax(scores)\n    test_split_USAirways.loc[indices[j], "pred"] = np.argmax(scores)\n    j+=1;\n    print(\'i\', end ="")'

In [44]:
#test_split_USAirways.to_csv("/kaggle/working/test_split_USAirways.csv")

In [45]:
'''for i in range(3):
    print(test_split_USAirways.loc[test_split_USAirways['labels']==1].shape[0])'''

"for i in range(3):\n    print(test_split_USAirways.loc[test_split_USAirways['labels']==1].shape[0])"

## For American Split

In [46]:
'''trainer = Trainer(model, args, train_dataset=tok_ds_american, eval_dataset=tok_eval_ds_american,
                  tokenizer=tokz, compute_metrics=compute_metrics)
trainer.train()'''

'trainer = Trainer(model, args, train_dataset=tok_ds_american, eval_dataset=tok_eval_ds_american,\n                  tokenizer=tokz, compute_metrics=compute_metrics)\ntrainer.train()'

In [47]:
#trainer.save_model("/kaggle/working/finetuned")

In [48]:
'''model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/finetuned")
test_split_american['pred']=-1
encoded_input = [tokz(x, return_tensors='pt') for x in test_split_american['text']]
model.to("cpu")
j=0
indices=test_split_american.index
for i in encoded_input:
    output=model(**i)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    test_split_american.loc[indices[j], "pred"] = np.argmax(scores)
    j+=1;
    print('i', end ="")'''

'model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/finetuned")\ntest_split_american[\'pred\']=-1\nencoded_input = [tokz(x, return_tensors=\'pt\') for x in test_split_american[\'text\']]\nmodel.to("cpu")\nj=0\nindices=test_split_american.index\nfor i in encoded_input:\n    output=model(**i)\n    scores = output[0][0].detach().numpy()\n    scores = softmax(scores)\n    test_split_american.loc[indices[j], "pred"] = np.argmax(scores)\n    j+=1;\n    print(\'i\', end ="")'

In [49]:
#test_split_american.to_csv("/kaggle/working/test_split_american.csv")

In [50]:
'''for i in range(3):
    print(test_split_american.loc[test_split_american['labels']==1].shape[0])'''

"for i in range(3):\n    print(test_split_american.loc[test_split_american['labels']==1].shape[0])"

Finetuning and recording the statistics for the 3 cross validation is done. And looking at the scores, we tune our hyperparameters. Now that hyperparameters are chosen ( The choosing of hyperparameters hasnt been shown in this notebook ) we move on to finetuning model with the entire dataset, and comparing our results with the test dataset that we havent yet tested upon.

## Finetuning the model with the entire training data ( after choosing optimum hyperparameters ) 

In [51]:
train_df

,tweet_id,labels,airline,text
0,567900433542488064,0,Southwest,"@user I am scheduled for the morning, 2 days a..."
1,569989168903819264,2,Southwest,@user seeing your workers time in and time out...
2,568089179520954368,2,United,@user Flew ORD to Miami and back and had grea...
3,568928195581513728,0,Southwest,@user @user that's horse radish 😤🐴
4,568594180014014464,0,United,@user so our flight into ORD was delayed becau...
...,...,...,...,...
10975,569934458364813313,1,American,@user followback
10976,568564006329434113,2,United,@user thanks for the help. Wish the phone reps...
10977,569643648910028801,0,US Airways,@user the. Worst. Ever. #dca #customerservice
10978,568864981917110272,0,US Airways,@user look! Another apology. DO NOT FLY @user


In [52]:
ds = Dataset.from_pandas(train_df)
eval_ds = Dataset.from_pandas(test_df)

In [53]:
tok_ds = ds.map(tokFunc, batched=True)
tok_eval_ds = eval_ds.map(tokFunc, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [54]:
trainer = Trainer(model, args, train_dataset=tok_ds, eval_dataset=tok_eval_ds,
                  tokenizer=tokz, compute_metrics=compute_metrics)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [55]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss,Accuracy
160,0.412500,0.533358,0.824249
320,0.281400,0.606954,0.828699


TrainOutput(global_step=344, training_loss=0.3410301028296005, metrics={'train_runtime': 158.5387, 'train_samples_per_second': 138.515, 'train_steps_per_second': 2.17, 'total_flos': 511730930598048.0, 'train_loss': 0.3410301028296005, 'epoch': 2.0})

In [60]:
!pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [61]:
from huggingface_hub import notebook_login
notebook_login()

In [62]:
trainer.save_model("/kaggle/working/FineTunedv1")
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jdhr/results_united/commit/78513fdae45bcbc18575a6d1348a4e42bae1467c', commit_message='End of training', commit_description='', oid='78513fdae45bcbc18575a6d1348a4e42bae1467c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
'''model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/FineTunedv1")
test_df['pred']=-1
encoded_input = [tokz(x, return_tensors='pt') for x in test_df['text']]
model.to("cpu")
j=0
indices=test_df.index
for i in encoded_input:
    output=model(**i)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    test_df.loc[indices[j], "pred"] = np.argmax(scores)
    j+=1'''

In [ ]:
'''test_df.to_csv("/kaggle/working/test_df.csv")'''

This finishes the finetuning of model. The statistics will be shown in the project report.